<a href="https://colab.research.google.com/github/johnathan2012/Programming-iOS-Book-Examples/blob/master/GPT4Dev_ch09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9 AI 助理開發實戰

## 9-1 看截圖就能生出類似網頁的小助理

In [ ]:
!pip install openai rich
from rich import print as pprint
from google.colab import userdata
from IPython.display import display, HTML
from openai import OpenAI
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
!git clone https://github.com/FlagTech/Assistant-API-Function.git assistant_function
import assistant_function as af
af.set_client(client)

Cloning into 'assistant_function'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 52 (delta 16), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (52/52), 14.04 KiB | 7.02 MiB/s, done.
Resolving deltas: 100% (16/16), done.


### 使用 GPT-4-Vision 建立網頁模板

In [ ]:
html_prompt = """
您是一位專業的 Tailwind 開發者。您會根據用戶提供的網頁截圖，使用 Tailwind、
HTML 和 JS 建構網頁。

-確保網頁的外觀與截圖完全一致。
-要特別注意文字顏色、字體大小、字體家族、內邊距、外邊距、邊框等。精確匹配顏色和尺寸。
-生成網頁大小需符合一般筆電螢幕。
-使用截圖中的準確文字。
-不要在程式碼中添加註解，例如 "<!-- 添加其他導航鏈接 -->" 或
"<!-- ... 其他城市項目 ... -->" 來代替完整的程式碼。請寫出完整的程式碼。
-根據需要重複元素以匹配截圖。例如，如果有 15 個項目，程式碼應該有 15 個項目。
不要留下像 "<!-- 重複每個新聞項目 -->" 這樣的註解，否則會出現問題。
不要背景圖，但其他項目（如天氣圖片等）請使用 Font Awesome 生成,
並在 alt 文本中包括詳細的圖片描述。

在使用的套件方面，
Tailwind：<script src="https://cdn.tailwindcss.com"></script>
字體 Font Awesome：
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/\
ajax/libs/font-awesome/5.15.3/css/all.min.css"></link>
請僅返回包含在 <html></html> 標籤中的完整程式碼。
不要在開頭或結尾處包含 markdown "" 或 "html"。
"""

In [ ]:
def html_generation(url):
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {"role": "system","content": html_prompt},
            {"role": "user","content": [
                {"type": "text","text": "請生成一個外觀與這張圖片"
                                        "完全一致的網頁程式碼"},
                {"type": "image_url","image_url": {"url": url}}],
            }],
        max_tokens=3000
    )
    display(HTML(response.choices[0].message.content))
    return response.choices[0].message.content

### 建立成 Function_calling

In [ ]:
tools_table = [{
    "function": html_generation, # 實際的函式
    "spec": {
        "name": "html_generation",
      "description": "使用圖片連結產生網頁程式碼,使用繁體中文",
      "parameters": {
        "type": "object",
        "properties": {
          "url": {"type": "string"}
        },
        "required": ["url"]
      }
    }
},]

In [ ]:
tools = [{"type": "code_interpreter"}]
for tool in tools_table:
    tools.append(
        {"type": "function", "function": tool['spec']}
    )
assistant = client.beta.assistants.create(
    name="網頁設計助理",
    instructions="你是一位網頁設計師, 能依照對話內容設計和修改出網頁, "
                 "請生成 HTML 檔",
    tools=tools,
    model="gpt-4-1106-preview",
)
assistant_id = assistant.id
# 新建討論串
thread = client.beta.threads.create()
thread_id = thread.id

In [ ]:
response = af.chat_with_functions('https://ppt.cc/fIxsSx'
                                  '請使用繁體中文, 最後請給我 HTML 檔',
                                  tools_table,
                                  thread_id, assistant_id)

-----------------------------------
|
處理中...
html_generation({'url': 'https://ppt.cc/fIxsSx'})


-----------------------------------
|


In [ ]:
for data in response:
    print(data.content[0].text.value)
af.show_html(response)

網頁設計已完成，並且已經存儲為 HTML 檔案。你可以通過以下連結下載該檔案：

[下載 weather_page.html](sandbox:/mnt/data/weather_page.html)


## 9-2 自動串接 API - 取得真正資料

### 串接 API 程式碼


這裡的網址與書上不同是由於 replit 已經在 2024/01/01 取消免費帳戶的 always on 功能, 所以無法作為可隨時自動啟動的 API 服務, 因此改用 Google 的 Apps Script 服務建立同樣功能的專案, 以下網址也更改成 Apps Script 專案的 API, 並且改用 GET 方法取得資料。

想要了解更多可以到 https://hackmd.io/_rMU5SRzSFOX4Sjk0uMnDQ?both 查看

In [ ]:
import requests
city = '臺北市'
response = requests.get(f'https://script.google.com/macros/s/AKfycbzmeU-mQXx7qjQSDjFCslQeT1OSU6HDRnRg9o3NmtZvD02DDhcO9RcK-K2oOn0ZigX5/exec?city={city}')
pprint(response.json())

{
    '臺北市': [
        {'日期': '2024-01-11', '天氣狀態': '晴時多雲', '最高溫': '17', '最低溫': '11'},
        {'日期': '2024-01-12', '天氣狀態': '晴時多雲', '最高溫': '21', '最低溫': '12'},
        {'日期': '2024-01-13', '天氣狀態': '晴時多雲', '最高溫': '20', '最低溫': '15'},
        {'日期': '2024-01-14', '天氣狀態': '多雲時晴', '最高溫': '23', '最低溫': '16'},
        {'日期': '2024-01-15', '天氣狀態': '多雲短暫雨', '最高溫': '18', '最低溫': '16'},
        {'日期': '2024-01-16', '天氣狀態': '多雲時陰短暫雨', '最高溫': '19', '最低溫': '16'}
    ]
}

### 製作天氣預報網頁

**第一步 輸入圖片連結：** <br>
https://ppt.cc/fIxsSx 請使用繁體中文, 最後請給我 HTML 檔

**第二步 製作下拉表單：** <br>
請增加一個下拉式表單並填入以下選項['宜蘭縣', '花蓮縣', '臺東縣', '澎湖縣', '金門縣','連江縣', '臺北市', '新北市', '桃園市', '臺中市', '臺南市','高雄市', '基隆市', '新竹縣', '新竹市', '苗栗縣', '彰化縣','南投縣', '雲林縣', '嘉義縣', '嘉義市', '屏東縣'],請勿使用`<!-- 其他城市選項 -->`

**第三步 串接 API 程式：** <br>
使用繁體中文, 並將以下 API 程式加入到 HTML 檔案並修改 HTML 程式碼, 當選取下拉式表單時請將值代入到 city 中, import requests response = requests.get(f'https://script.google.com/macros/s/AKfycbzmeU-mQXx7qjQSDjFCslQeT1OSU6HDRnRg9o3NmtZvD02DDhcO9RcK-K2oOn0ZigX5/exec?city={city}') # 返回資料用 response.json()

參考資料範例:{ '嘉義市': [ {'天氣狀態': '多雲', '日期': '2023-12-19', '最低溫': '17', '最高溫': '27'}, {'天氣狀態': '多雲', '日期': '2023-12-20', '最低溫': '15', '最高溫': '24'}, {'天氣狀態': '晴時多雲', '日期': '2023-12-21', '最低溫': '14', '最高溫': '18'}, {'天氣狀態': '晴時多雲', '日期': '2023-12-22', '最低溫': '10', '最高溫': '18'}, {'天氣狀態': '陰時多雲', '日期': '2023-12-23', '最低溫': '11', '最高溫': '21'}, {'天氣狀態': '晴時多雲', '日期': '2023-12-24', '最低溫': '13', '最高溫': '21'}, {'天氣狀態': '晴時多雲', '日期': '2023-12-25', '最低溫': '13', '最高溫': '22'} ] }

最後可以動態顯示出 API 返回的資料(包含圖示), 請勿使用//省略程式碼, 請提供完整程式碼, 最後請給我 HTML 檔


In [ ]:
while True:
    question = input('請輸入要求：')
    if not question.strip():
        break
    response = af.chat_with_functions(question, tools_table,
                                      thread_id, assistant_id)
    for data in response:
        print(data.content[0].text.value)

    af.show_html(response)

請輸入要求：https://ppt.cc/fIxsSx 請使用繁體中文, 最後請給我 HTML 檔
-----------------------------------
|
處理中...
html_generation({'url': 'https://ppt.cc/fIxsSx'})


-----------------------------------
|
網頁已經設計完畢，並且已將HTML檔案保存。你可以點擊以下鏈接下載HTML檔案：

[下載天氣網頁HTML檔案](sandbox:/mnt/data/weather_page.html)


請輸入要求：請增加一個下拉式表單並填入以下選項['宜蘭縣', '花蓮縣', '臺東縣', '澎湖縣', '金門縣','連江縣', '臺北市', '新北市', '桃園市', '臺中市', '臺南市','高雄市', '基隆市', '新竹縣', '新竹市', '苗栗縣', '彰化縣','南投縣', '雲林縣', '嘉義縣', '嘉義市', '屏東縣'],請勿使用<!-- 其他城市選項 -->
-----------------------------------
|
我已經增加了一個包含指定選項的下拉式表單到網頁中，並且已將更新後的HTML檔案儲存。您可以通過下面的鏈接下載新的HTML檔案：

[下載包含下拉表單的天氣網頁HTML檔案](sandbox:/mnt/data/weather_page_with_dropdown.html)


請輸入要求：使用繁體中文, 並將以下 API 程式加入到 HTML 檔案並修改 HTML 程式碼, 當選取下拉式表單時請將值代入到 city 中, import requests response = requests.get(f'https://script.google.com/macros/s/AKfycbzmeU-mQXx7qjQSDjFCslQeT1OSU6HDRnRg9o3NmtZvD02DDhcO9RcK-K2oOn0ZigX5/exec?city={city}') # 返回資料用 response.json()  參考資料範例:{ '嘉義市': [ {'天氣狀態': '多雲', '日期': '2023-12-19', '最低溫': '17', '最高溫': '27'}, {'天氣狀態': '多雲', '日期': '2023-12-20', '最低溫': '15', '最高溫': '24'}, {'天氣狀態': '晴時多雲', '日期': '2023-12-21', '最低溫': '14', '最高溫': '18'}, {'天氣狀態': '晴時多雲', '日期': '2023-12-22', '最低溫': '10', '最高溫': '18'}, {'天氣狀態': '陰時多雲', '日期': '2023-12-23', '最低溫': '11', '最高溫': '21'}, {'天氣狀態': '晴時多雲', '日期': '2023-12-24', '最低溫': '13', '最高溫': '21'}, {'天氣狀態': '晴時多雲', '日期': '2023-12-25', '最低溫': '13', '最高溫': '22'} ] }  最後可以動態顯示出 API 返回的資料(包含圖示), 請勿使用//省略程式碼, 請提供完整程式碼, 最後請給我 HTML 檔
-----------------------------------
|
我已經將API程序加入到HTML文件中，並且修改了HTML程式碼，以便當選取下拉式表單時該值能夠傳入到 `city` 變量中。同時程式碼也支援動態顯示API返回的資料（包括圖示）。現在您可以通過以下鏈接下載包含API呼叫的HTML文件：

[下載包含API呼叫的天氣網頁HTML檔案](sandbox:/mnt/data/weat

請輸入要求：


## 9-3 SQL 資料庫查詢助理

### 下載資料

In [ ]:
import yfinance as yf
stock = ["2330.TW","2317.TW"]
df = yf.download(stock,period='5y')
df.tail()

[*********************100%%**********************]  2 of 2 completed


Adj Close           Close            High             Low          \
             2317.TW 2330.TW 2317.TW 2330.TW 2317.TW 2330.TW 2317.TW 2330.TW   
Date                                                                           
2023-12-26     104.0   586.0   104.0   586.0   104.0   586.0   103.5   582.0   
2023-12-27     103.5   592.0   103.5   592.0   104.0   592.0   103.0   586.0   
2023-12-28     104.0   593.0   104.0   593.0   104.0   593.0   103.0   589.0   
2023-12-29     104.5   593.0   104.5   593.0   105.0   593.0   103.5   589.0   
2024-01-02     105.0   593.0   105.0   593.0   105.0   593.0   104.0   589.0   

              Open            Volume            
           2317.TW 2330.TW   2317.TW   2330.TW  
Date                                            
2023-12-26   104.0   583.0   9920950  16094308  
2023-12-27   104.0   587.0  18875621  33401336  
2023-12-28   103.5   592.0  23057092  25514849  
2023-12-29   104.0   589.0  25969421  18416318  
2024-01-02   104.5   590.0  19865654  26409497

### 建立 SQL 資料庫

In [ ]:
import sqlite3
conn = sqlite3.connect("stock.db")
cursor = conn.cursor()

In [ ]:
stock_name_list=['db_2330', 'db_2317']
for name in stock_name_list:
    cursor.execute(f"""Create table if not exists {name} (
        Date TEXT PRIMARY KEY,
        Open REAL,
        High REAL,
        Low REAL,
        Close REAL,
        Adj_Close REAL,
        Volume INTEGER
    ); """)

In [ ]:
import pandas as pd
for i, name in zip(stock, stock_name_list):
    stock_df = df.xs(i, level=1, axis=1)
    stock_df.to_sql(name, con=conn,
                    if_exists='replace',
                    index=True)

In [ ]:
table_data = cursor.execute("Select * from db_2330 limit 5")
for data in table_data:
    print(data)

('2019-01-02 00:00:00', 191.67147827148438, 219.5, 226.5, 219.0, 226.5, 32900482)
('2019-01-03 00:00:00', 188.17857360839844, 215.5, 218.0, 214.0, 214.0, 34087620)
('2019-01-04 00:00:00', 181.62945556640625, 208.0, 211.5, 206.5, 211.5, 65943521)
('2019-01-07 00:00:00', 185.99554443359375, 213.0, 214.0, 211.0, 212.0, 35442176)
('2019-01-08 00:00:00', 184.2490997314453, 211.0, 212.5, 210.0, 212.0, 22694481)


In [ ]:
table_structure = cursor.execute(
    "SELECT sql FROM sqlite_master WHERE type='table';")
for statement in table_structure:
    print(statement[0])

CREATE TABLE "db_2330" (
"Date" TIMESTAMP,
  "Adj Close" REAL,
  "Close" REAL,
  "High" REAL,
  "Low" REAL,
  "Open" REAL,
  "Volume" INTEGER
)
CREATE TABLE "db_2317" (
"Date" TIMESTAMP,
  "Adj Close" REAL,
  "Close" REAL,
  "High" REAL,
  "Low" REAL,
  "Open" REAL,
  "Volume" INTEGER
)


### 建立相關函式

In [ ]:
import csv
from bs4 import BeautifulSoup
import pandas as pd
import os

def find_stock_info(search_term):
    with open('stock.csv', mode='r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            if row['股號'] == search_term or row['股名'] == search_term:
                return row
    return '無結果'

def search_stock(stock):
    if os.path.exists('stock.csv'):
        row = find_stock_info(stock)
        return row
    else:
        data=[]
        response=requests.get(
            'https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
        url_data=BeautifulSoup(response.text, 'html.parser')
        stock_company=url_data.find_all('tr')
        for i in stock_company[2:]:
            j=i.find_all('td')
            l=j[0].text.split('\u3000')
            if len(l[0].strip()) == 4:
                stock_id,stock_name = l
                data.append([stock_id.strip(),stock_name])
            else:
                break
        df = pd.DataFrame(data, columns=['股號','股名'])
        # 建立上市股票公司 CSV 檔
        df.to_csv('stock.csv',index=False)
        row = find_stock_info(stock)
        return row

def get_db_schema():
        conn = sqlite3.connect('stock.db')
        cursor = conn.cursor()
        cursor.execute(
            "SELECT sql FROM sqlite_master WHERE type='table';")
        create_statements = cursor.fetchall()
        conn.close()
        return '\n\n'.join(
            [statement[0] for statement in create_statements])

def run_sql_query(query):
        conn = sqlite3.connect('stock.db')
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        conn.close()
        return '\n'.join([str(result) for result in results])

### 建立 SQL 助理

In [ ]:
tools_table = [{
    'function': get_db_schema,
    'spec': {
         "name": "get_db_schema",
      "description": "取得 sqlite 資料庫的架構",
      "parameters": {
        "type": "object",
        "properties": {}
      }
    }
},{
    "function": run_sql_query,
    "spec": {
        "name": "run_sql_query",
      "description": "在 sqlite 資料庫上執行 SQL 查詢",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {"type": "string",
                    "description": "SQL 語句"}
        },
        "required": ["query"]
      }
    }
},{
    "function": search_stock,
    "spec": {
        "name": "search_stock",
      "description": "如果不知道股票名稱或股票代碼可以使用這個工具",
      "parameters": {
        "type": "object",
        "properties": {
          "stock": {"type": "string",
                    "description": "為股票名稱或股票代碼"}
        },
        "required": ["stock"]
      }
    }
},]

In [ ]:
tools = [{"type": "code_interpreter"}]
for tool in tools_table:
    tools.append(
        {"type": "function", "function": tool['spec']}
    )
assistant = client.beta.assistants.create(
  name="SQL 助理",
  instructions="您是 SQL 專家, 用戶向您詢問有關 sqlite 資料庫的問題。"
               "首先請取得資料庫的架構，表格名稱都是使用 db_ + 股票代碼, "
               "然後才產生 SQL 查詢來回答問題。",
  model="gpt-4-1106-preview",
  tools=tools
)
assistant_id = assistant.id
thread = client.beta.threads.create()
thread_id = thread.id

台積電近一周的股價為何？

找出台積電和鴻海三天內的股價並計算漲幅

In [ ]:
while True:
    question = input('請輸入要求：')
    if not question.strip():
        break
    response = af.chat_with_functions(question, tools_table,
                                      thread_id, assistant_id)
    for data in response:
        print(data.content[0].text.value)

請輸入要求：台積電近一周的股價為何？
-----------------------------------
|
處理中...
search_stock({'stock': '台積電'})
-----------------------------------
|
處理中...
get_db_schema({})
-----------------------------------
|
處理中...
run_sql_query({'query': "SELECT Date, Close FROM db_2330 WHERE Date > datetime('now', '-7 days')"})
-----------------------------------
|
台積電近一周的股價資訊如下：

- 2023-12-27: 收盤價為 592.0
- 2023-12-28: 收盤價為 593.0
- 2023-12-29: 收盤價為 593.0
- 2024-01-02: 收盤價為 593.0

請注意，正確的日期應該要是最近七天內的資訊，但這些資料看起來是未來的日期。請確認您的系統日期設定是否正確。如果需要最新的股市資訊，可能需要從股市的即時資料來源獲取。
請輸入要求：找出台積電和鴻海三天內的股價並計算漲幅
-----------------------------------
|
處理中...
search_stock({'stock': '鴻海'})
-----------------------------------
|
處理中...
run_sql_query({'query': "SELECT Date, Close FROM db_2330 WHERE Date > datetime('now', '-3 days')"})
run_sql_query({'query': "SELECT Date, Close FROM db_2317 WHERE Date > datetime('now', '-3 days')"})
-----------------------------------
|
台積電和鴻海過去三天的股價資訊如下：

- 台積電 (2330)
  - 2024-01-02: 收盤價為 593.0
- 鴻海 (2317)
  -